In [1]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install pytrends
!pip install pandas
import pandas as pd
import datetime      
import csv
from pytrends.request import TrendReq

In [11]:
# Create functions (can remove or add accordingly)
def getFileName(type, func, word, ts):
	if type == 'figure' :
		txt = type + '_' + func + '_' + word + '_{}.png'
	elif type == 'CSV' :
		txt = type + '_' + func + '_' + word + '_{}.csv'
	fileName  = txt.format(ts)
	return fileName

def time_analysis(source, ts):
	figName = getFileName('figure', 'analysis_over_time_SG', source, ts)
	csvName = getFileName('CSV', 'analysis_over_time_SG', source, ts)
	df = pytrend.interest_over_time()
	df = df.drop(labels=['isPartial'],axis='columns', errors = 'ignore')
	df.to_csv('/content/drive/MyDrive/Official Project Folder/Tasks/Google Trends #Task 1_6/Codes' + csvName, encoding='utf_8_sig') #Can Change Path Where Relevant 
	image = df.plot(title = source + ' searchs in 2019 to June 2021 on Google Trends Singapore') 
	fig = image.get_figure()
	fig.savefig('/content/drive/MyDrive/Official Project Folder/Tasks/Google Trends #Task 1_6/Codes' + figName)

def suggestions_analysis(source, ts):
	figName = getFileName('figure', 'suggestions_SG', source, ts)
	csvName = getFileName('CSV', 'suggestions_SG', source, ts)
	keywords = pytrend.suggestions(keyword=source)
	df = pd.DataFrame(keywords)
	df = df.drop(columns='mid', errors = 'ignore')
	df.to_csv('/content/drive/MyDrive/Official Project Folder/Tasks/Google Trends #Task 1_6/Codes' + csvName, encoding='utf_8_sig')

def related_topics_analysis(source, ts):
	figName = getFileName('figure', 'related_topics_SG', source, ts)
	csvName = getFileName('CSV', 'related_topics_SG', source, ts)
	related_topics = pytrend.related_topics()
	with open('/content/drive/MyDrive/Official Project Folder/Tasks/Google Trends #Task 1_6/Codes' + csvName, 'w') as f:
		for key in related_topics.keys():
			f.write("%s,%s\n"%(key,related_topics[key]))
 

In [22]:
# Loading input data
df = wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Mjk9g99XqNXQeGkJxDcbAdeLLZ8Wt3rflsXTF1wWggc/edit#gid=0') #Change Input File 
sheet = wb.sheet1
keywords = list(sheet.col_values(4))
date_from = sheet.acell('B7').value
date_to = sheet.acell('B8').value
geo = sheet.acell('B10').value

In [23]:
# Function to loop through the keyword and get information
# Run through keywords individually to create individual csv and figs 
def Main(keywords, ts):
	global pytrend 
	pytrend = TrendReq(tz = -480)
	for word in keywords:
		print('\n' + word)
		pytrend.build_payload(kw_list= [word], timeframe = timeframe, geo = geo)
		try:
			time_analysis(word, ts)
		except: # catch *all* exceptions
			e = sys.exc_info()[0]
			print( "Error during time analysis execution: %s" % e )
		else:
	  		print("Time analysis completed successfully") 
		try:
			suggestions_analysis(word, ts)
		except: # catch *all* exceptions
			e = sys.exc_info()[0]
			print( "Error during suggestions analysis execution: %s" % e )
		else:
	  		print("Suggestions analysis completed successfully")
     

In [ ]:
# Setting some parameteres
ts = int(datetime.datetime.now().timestamp())
timeframe = date_from + " " + date_to

# Run code
Main(keywords, ts)

References: 

https://www.honchosearch.com/blog/seo/how-to-use-python-pytrends-to-automate-google-trends-data/
https://github.com/OmdenaAI/omdena-singapore-covid-health/tree/main/original/Data_collection/GoogleTrends